# 0.Load data from netCDF

In [ ]:
dataset = xr.open_dataset('test.nc')

# 1.List band

In [ ]:
list(dataset.data_vars.keys())

# 2.Select Band

In [ ]:
green = dataset.green

In [ ]:
green.plot(col='time', col_wrap=10)

# 3.Use where

In [ ]:
autumn = dataset.green.where(dataset.pixel_qa == 322)

In [ ]:
autumn_valid = autumn.where(autumn != autumn.attrs['nodata'])

In [ ]:
autumn_valid.plot(col='time', col_wrap=10)

# 4.NDVI example

In [ ]:
red = combined_dataset.red.where(combined_dataset.red != combined_dataset.red.attrs['nodata']).where(combined_dataset.pixel_qa == 322)
nir = combined_dataset.nir.where(combined_dataset.nir != combined_dataset.nir.attrs['nodata']).where(combined_dataset.pixel_qa == 322)

ndvi = ((nir - red) / (nir + red)).where(ndvi <= 1).where(ndvi >= -1)

In [ ]:
ndvi.plot(col='time', col_wrap=10)

In [ ]:
cloud_free = combined_dataset.pixel_qa.where(combined_dataset.pixel_qa != combined_dataset.pixel_qa.attrs['nodata']).where(combined_dataset.pixel_qa == 322) / 322
mostly_cloud_free = cloud_free.sum(dim=('latitude','longitude')) > (0.75 * cloud_free.size / cloud_free.time.size)
mostly_good_ndvi = ndvi.where(mostly_cloud_free).dropna('time', how='all')

In [ ]:
mostly_good_ndvi.plot(col='time', col_wrap=10)

In [ ]:
mostly_good_ndvi.median(dim='time').plot()

In [ ]:
mostly_good_ndvi.max(dim='time').plot()

In [ ]:
mostly_good_ndvi.std(dim='time').plot()

In [ ]:
mostly_good_ndvi.sel(latitude=24.958132, longitude=121.126085, method='nearest').plot()

# 5.Cloud free example

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
data = dataset
cloud_free_green = data.green.where(data.green > 0).where(data.pixel_qa == 322)
#cloud_free.plot(col='time', col_wrap=3)
cloud_free_red = data.red.where(data.red > 0).where(data.pixel_qa == 322)
cloud_free_blue = data.blue.where(data.blue > 0).where(data.pixel_qa == 322)
cloud_free_nir = data.nir.where(data.nir > 0).where(data.pixel_qa == 322)
cloud_free_swir1 = data.swir1.where(data.swir1 > 0).where(data.pixel_qa == 322)
cloud_free_swir2 = data.swir2.where(data.swir2 > 0).where(data.pixel_qa == 322)

In [ ]:
combine_green = cloud_free_green.median(dim='time')
combine_red = cloud_free_red.median(dim='time')
combine_blue = cloud_free_blue.median(dim='time')
combine_nir = cloud_free_nir.median(dim='time')
combine_swir1 = cloud_free_swir1.median(dim='time')
combine_swir2 = cloud_free_swir2.median(dim='time')

In [ ]:
combine_green.plot()

In [ ]:
max_val = combine_green.max(['latitude', 'longitude'])
# 然後找最大的scale
scaled = (combine_green / max_val)
# 設定scaled

from matplotlib import pyplot as plt
#plt.imshow(scaled.isel(time=19))
plt.figure(figsize = (100,19))
plt.imshow(scaled, interpolation='nearest')

In [ ]:
combine_red.plot()

In [ ]:
combine_blue.plot()

In [ ]:
import xarray as xr
test = xr.Dataset({})
for band in desired_bands:
    if band != 'pixel_qa':
        temp1 = data.get([band]).where(data.get([band]) > 0).where(data.pixel_qa == 322).median(dim='time')
        test = test.merge(temp1)
test

In [ ]:
from utils.dc_utilities import write_geotiff_from_xr, write_png_from_xr

measurements = desired_bands.copy()
measurements.remove('pixel_qa')
measurements
write_geotiff_from_xr("/home/localuser/Datacube/data_cube_notebooks/NTUF_Hsing-Yu/taipei_cloud_remove_new.tiff", test, measurements, nodata=0, crs="EPSG:4326")

In [ ]:
combine = combine_green.to_dataset(name = "green")
combine = combine.merge(combine_red.to_dataset(name = "red"))
combine = combine.merge(combine_blue.to_dataset(name = "blue"))

In [ ]:
combine

In [ ]:
rbg = combine.to_array(dim='color').transpose('latitude', 'longitude', 'color')

In [ ]:
fake_saturation = 3000
# 這個是用來當作影像值得上限，超過就用3000取代
clipped_visible = rbg.where(rbg < fake_saturation).fillna(fake_saturation)
# 上面這邊就是這樣做，透過fillna()函式可以填入值
max_val = clipped_visible.max(['latitude', 'longitude'])
# 然後找最大的scale
scaled = (clipped_visible / max_val)
# 設定scaled

from matplotlib import pyplot as plt
#plt.imshow(scaled.isel(time=19))
plt.figure(figsize = (100,19))
plt.imshow(scaled, interpolation='nearest')
# 最後就印出來，記得要import該import的東西

In [ ]:
from utils.dc_utilities import write_geotiff_from_xr, write_png_from_xr
measurements = ['green','red','blue']
write_geotiff_from_xr("/home/localuser/Datacube/data_cube_notebooks/NTUF_Hsing-Yu/2014-2013_color-only.tiff", combine, measurements, nodata=0, crs="EPSG:4326")

In [ ]:
#這個製出的圖顏色會怪怪的
write_png_from_xr("/home/localuser/Datacube/data_cube_notebooks/NTUF_Hsing-Yu/2015taipei_cloud_remove.png", combine, ['green','red','blue'], scale=[[255,0], [255,0], [255,0]])

In [ ]:
#替代方案，用matplotlib製成png檔案
plt.imsave("/home/localuser/Datacube/data_cube_notebooks/NTUF_Hsing-Yu/2015taipei_cloud_remove2.png", scaled)